In [1]:
# acquisition_and_analysis.py
import tdwf, time, csv
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch
from datetime import datetime

# --- CONFIG ---
SAD = 0x68
ACCEL_SCALE = 16384.0   # LSB/g for ±2g
GYRO_SCALE  = 131.0     # LSB/(°/s) for ±250 °/s

ACCEL_XOUT_H = 0x3B
TEMP_OUT_H   = 0x41
GYRO_XOUT_H  = 0x43
# helper to read word (as in Task 5)
def read_word(dev, sub):
    dev.write([sub])
    dev.read(2)
    high, low = dev.vals[0], dev.vals[1]
    val = (high << 8) | low
    if val & 0x8000:
        val -= 65536
    return val

# --- Connect to AD2 ---
ad2 = tdwf.AD2()
ad2.vdd = 3.3
ad2.power(True)
i2c = tdwf.I2Cbus(ad2.hdwf)
mpu = tdwf.I2Cdevice(ad2.hdwf, SAD)

# Optional: wake-up (PWR_MGMT_1 = 0x6B)
PWR_MGMT_1 = 0x6B
mpu.write([PWR_MGMT_1, 0x00])
time.sleep(0.1)

# Acquisition parameters
fs = 200.0        # Hz
dt = 1.0/fs
duration = 120.0  # seconds
n_samples = int(duration * fs)

# Pre-run bias estimation (5 s)
print("Bias estimation: hold sensor still for 5 s...")
bias_samples = []
for _ in range(int(5*fs)):
    ax = read_word(mpu, ACCEL_XOUT_H) / ACCEL_SCALE
    gx = read_word(mpu, GYRO_XOUT_H) / GYRO_SCALE
    bias_samples.append((ax,gx))
    time.sleep(dt)
bias_arr = np.array(bias_samples)
bias_ax = np.mean(bias_arr[:,0])
bias_gx = np.mean(bias_arr[:,1])
print("bias_ax", bias_ax, "bias_gx", bias_gx)

# Main acquisition
fname = f"mpu_run_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
with open(fname, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["t","ax","ay","az","gx","gy","gz","temp"])
    t0 = time.time()
    for i in range(n_samples):
        t = time.time() - t0
        ax_raw = read_word(mpu, ACCEL_XOUT_H)
        ay_raw = read_word(mpu, ACCEL_XOUT_H+2)
        az_raw = read_word(mpu, ACCEL_XOUT_H+4)
        temp_raw = read_word(mpu, TEMP_OUT_H)
        gx_raw = read_word(mpu, GYRO_XOUT_H)
        gy_raw = read_word(mpu, GYRO_XOUT_H+2)
        gz_raw = read_word(mpu, GYRO_XOUT_H+4)

        ax = ax_raw / ACCEL_SCALE
        ay = ay_raw / ACCEL_SCALE
        az = az_raw / ACCEL_SCALE
        gx = gx_raw / GYRO_SCALE
        gy = gy_raw / GYRO_SCALE
        gz = gz_raw / GYRO_SCALE
        temp_c = temp_raw / 340.0 + 36.5

        writer.writerow([t,ax,ay,az,gx,gy,gz,temp_c])
        # periodic print
        if i % int(fs*1) == 0:  # every 1 second
            print(f"t={t:.1f}s ax={ax:.3f}g az={az:.3f}g gx={gx:.2f}deg/s T={temp_c:.2f}")
        # wait
        time.sleep(dt - 0.001)  # small adjust

print("Done. File:", fname)
ad2.close()

# --- ANALYSIS (load the file) ---
data = np.loadtxt(fname, delimiter=",", skiprows=1)
t = data[:,0]
ax = data[:,1]; ay = data[:,2]; az = data[:,3]
gx = data[:,4]; gy = data[:,5]; gz = data[:,6]
temp = data[:,7]

def analyze_channel(x, fs, label):
    # basic stats
    mean = np.mean(x); std = np.std(x)
    print(f"{label}: mean={mean:.6f}, std={std:.6f}")
    # histogram
    plt.figure(); plt.hist(x, bins=80, density=True)
    plt.title(f"{label} histogram (mean={mean:.3e}, std={std:.3e})")
    # PSD Welch
    f, Pxx = welch(x, fs=fs, window='hann', nperseg=2048, noverlap=1024, scaling='density')
    plt.figure(); plt.semilogy(f, Pxx); plt.title(f"{label} PSD (Welch)")
    plt.xlabel('Hz'); plt.ylabel('PSD')
    # autocorr (short)
    acf = np.correlate(x - mean, x - mean, mode='full')
    acf = acf[acf.size//2:] / acf[acf.size//2]
    plt.figure(); plt.plot(np.arange(len(acf))/fs, acf); plt.title(f"{label} ACF")
    return mean, std, f, Pxx

analyze_channel(az, fs, 'acc_z')
analyze_channel(gz, fs, 'gyro_z')
plt.show()


Digilent WaveForms SDK versione 3.22.2
Dispositivo #1 [SN:210321B5D136, hdwf=1] connesso!
Configurazione #1
Bus I2C pronto...
Bias estimation: hold sensor still for 5 s...
bias_ax -0.738181396484375 bias_gx 1.5144351145038168
t=0.0s ax=-0.737g az=0.027g gx=-2.29deg/s T=27.04
t=7.4s ax=-0.735g az=0.010g gx=1.38deg/s T=27.14
t=14.7s ax=-0.742g az=0.005g gx=1.41deg/s T=27.09
t=21.9s ax=-0.737g az=0.002g gx=1.52deg/s T=27.32
t=29.2s ax=-0.741g az=0.013g gx=1.56deg/s T=27.23
t=36.5s ax=-0.742g az=0.010g gx=1.47deg/s T=27.32
t=43.7s ax=-0.742g az=0.012g gx=1.47deg/s T=27.42
t=51.3s ax=-0.740g az=0.006g gx=1.51deg/s T=27.42
t=58.7s ax=-0.744g az=0.005g gx=1.66deg/s T=27.42
t=66.2s ax=-0.745g az=0.008g gx=1.66deg/s T=27.37
t=73.7s ax=-0.743g az=0.015g gx=1.59deg/s T=27.61
t=81.2s ax=-0.746g az=0.015g gx=1.54deg/s T=27.51
t=88.8s ax=-0.736g az=0.017g gx=1.48deg/s T=27.61
t=96.3s ax=-0.746g az=0.011g gx=1.52deg/s T=27.56
t=103.8s ax=-0.742g az=0.004g gx=1.48deg/s T=27.65
t=111.1s ax=-0.744g az=0

KeyboardInterrupt: 